<a href="https://colab.research.google.com/github/demyanchuk-nestor/AI_for_Medical_Prognosis/blob/master/Week-2/C2W2_A1_Risk Models Using Tree-based Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/demyanchuk-nestor/AI_for_Medical_Prognosis
%cd 'AI_for_Medical_Prognosis/Week-2'

# Моделі ризику з використанням дерева рішень

Welcome to the second assignment of Course 2!

## Зміст
1. Імпорт пакетів
2. Завантажте набір даних
3. Дослідіть набір даних
4. Робота з відсутніми даними.
Вправа 1
5. Дерева рішень.
Вправа 2
6. Випадкові ліси.
Вправа 3
7. Імпутація
8. Аналіз помилок.
Вправа 4
9. Підходи імпутації.
Вправа 5
Вправа 6
10. Порівняння
11. Пояснення: SHAP

У цьому завданні ви отримаєте досвід роботи з моделями на основі дерева. Дерево ухвалення рішень (також можуть називатися деревами класифікацій або регресійними деревами) — використовується в галузі статистики та аналізу даних для прогнозних моделей. Структура дерева містить такі елементи: «листя» і «гілки». Дерева рішень широко використовуються в інтелектуальному аналізі даних. Мета полягає в тому, щоб створити модель, яка прогнозує значення цільової змінної на основі декількох змінних на вході. У даній роботі прогнозується 10-річний ризик смерті людей за набором епідеміологічних даних NHANES I (детальний опис цього набору даних можна знайти на веб-сайті CDC). Це складне завдання та чудова перевірка методів машинного навчання, які ми вивчаємо.

Виконуючи завдання, ви дізнаєтеся про:

Роботу з відсутніми даними

Повний аналіз випадку.

Імпутацію

Дерева рішень

Оцінку.

Регуляризацію.

Випадкові ліси

Гіперпараметричне нала.

<a name='1'></a>
## 1. Імпорт пакетів
Спочатку ми імпортуємо всі звичайні пакети, необхідні для цього завдання.

shap — це бібліотека, яка пояснює прогнози, зроблені моделями машинного навчання.

sklearn — одна з найпопулярніших бібліотек машинного навчання.

itertools дозволяє нам зручно маніпулювати ітерованими об’єктами, такими як списки.

pydotplus використовується разом з IPython.display.

Image для візуалізації графових структур, таких як дерева рішень.

numpy — базовий пакет для наукових обчислень на Python.

pandas — це те, що ми використовуватимемо для маніпулювання нашими даними.

seaborn — це бібліотека графіків, яка має кілька зручних функцій для візуалізації відсутніх даних.

matplotlib — це бібліотека для побудови графіків.

In [ ]:
!pip install shap==0.42.0 pydotplus six lifelines dill
!pip install typing-extensions==4.4.0
!pip install graphviz
!pip install matplotlib==3.5.1

In [ ]:
import shap
import sklearn
import itertools
import pydotplus
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image

from sklearn.tree import export_graphviz
from six import StringIO
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

# We'll also import some helper functions that will be useful later on.
from util import load_data, cindex

<a name='2'></a>
## 2. Завантажте набір даних

Запустіть наступну клітинку, щоб завантажити набір епідеміологічних даних NHANES I.

Цей набір даних містить різні характеристики лікарняних пацієнтів, а також їхні результати, тобто чи померли вони протягом 10 років.

In [ ]:
X_dev, X_test, y_dev, y_test = load_data(10)

Набір даних було розділено на набір для розробки (або набір для розробників), який ми будемо використовувати для розробки наших моделей ризику, і набір для тестування, який ми будемо використовувати для тестування наших моделей.

Далі ми розділили набір розробників на набір для навчання та перевірки відповідно, щоб навчити та налаштувати наші моделі, використовуючи розподіл 75/25 (зверніть увагу, що ми встановили випадковий стан, щоб зробити цей розподіл повторюваним).

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=10)

<a name='3'></a>
## 3. Дослідіть набір даних
Першим кроком є ​​ознайомлення з даними. Запустіть наступну клітинку, щоб отримати розмір вашого навчального набору, і подивіться на невеликий зразок.
У набір даних входить: Age - вік , Diastolic BP - , Poverty index - , Rase - склад крові, Red blood cells - еритроцити , Sedimentation rate - швидкість згортання, Serum Albumin - сироватковий альбумін, Serum Cholesterol - холестерин, Serum Iron - залізо, Serum Magnesium - магній, Serum Protein - білок, Sex - стать, Sistolic BP - систолічний АТ, TIBC - , TS - , White blood cells - лейкоцити, BMI - , Pulse pressure - пульс.

In [ ]:
print("X_train shape: {}".format(X_train.shape))
X_train.head()

Наші результати дослідження будуть залежати від того, чи помер пацієнт протягом 10 років. Запустіть наступну клітинку, щоб побачити цільовий ряд даних.

In [ ]:
y_train.head(10)

Використовуйте наступну клітинку, щоб розглянути окремі випадки та ознайомитися з особливостями.

In [ ]:
i = 10
print(X_train.iloc[i,:])
print("\nDied within 10 years? {}".format(y_train.loc[y_train.index[i]]))

<a name='4'></a>
## 4. Робота з відсутніми даними
Дивлячись на наші дані в X_train, ми бачимо, що деякі дані відсутні: деякі значення у виводі попередньої комірки позначені як NaN («не число»).

Відсутність даних є звичайним явищем під час аналізу даних, що може бути спричинено різними причинами, такими як несправність вимірювального приладу, небажання або нездатність пацієнтів (респондентів) надати інформацію та помилки в процесі збору даних.

Давайте розглянемо шаблон відсутніх даних. seaborn є альтернативою matplotlib, яка має деякі зручні функції побудови графіків для аналізу даних. Ми можемо використовувати функцію його карти, щоб легко візуалізувати шаблон відсутніх даних.

Запустіть клітинку нижче, щоб побудувати відсутні дані:

In [ ]:
sns.heatmap(X_train.isnull(), cbar=False)
plt.title("Training")
plt.show()

sns.heatmap(X_val.isnull(), cbar=False)
plt.title("Validation")
plt.show()

Для кожної функції, представленої у вигляді стовпця, наявні значення відображаються чорним кольором, а відсутні значення встановлюються світлим кольором.

На цьому графіку ми бачимо, що для систолічного артеріального тиску (систолічного АТ) бракує багатьох значень.

<a name='Ex-1'></a>
### Вправа 1
У клітинці нижче напишіть функцію для обчислення частки випадків із відсутніми даними. Це допоможе нам вирішити, як ми оброблятимемо ці відсутні дані в майбутньому.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> The <code>pandas.DataFrame.isnull()</code> method is helpful in this case.</li>
    <li> Use the <code>pandas.DataFrame.any()</code> method and set the <code>axis</code> parameter.</li>
    <li> Divide the total number of rows with missing data by the total number of rows. Remember that in Python, <code>True</code> values are equal to 1.</li>
</ul>
</p>

In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def fraction_rows_missing(df):
    '''
    Return percent of rows with any missing
    data in the dataframe.

    Input:
        df (dataframe): a pandas dataframe with potentially missing data
    Output:
        frac_missing (float): fraction of rows with missing data
    '''
    ### START CODE HERE (REPLACE 'Pass' with your 'return' code) ###
    return sum(df.isnull().any(axis=1)) / len(df)
    ### END CODE HERE ###

Перевірте свою функцію, запустивши клітинку нижче.

In [ ]:
df_test = pd.DataFrame({'a':[None, 1, 1, None], 'b':[1, None, 0, 1]})
print("Example dataframe:\n")
print(df_test)

print("\nComputed fraction missing: {}, expected: {}".format(fraction_rows_missing(df_test), 0.75))
print(f"Fraction of rows missing from X_train: {fraction_rows_missing(X_train):.3f}")
print(f"Fraction of rows missing from X_val: {fraction_rows_missing(X_val):.3f}")
print(f"Fraction of rows missing from X_test: {fraction_rows_missing(X_test):.3f}")

Ми бачимо, що в наших наборах длям навчання та перевірки відсутні значення, але, для нашого тестового набору ми використаємо повні випадки.

На першому етапі ми почнемо з аналізу повного випадку, видаливши всі рядки з відсутніми даними. Запустіть наступну клітинку, щоб вилучити ці рядки з нашого курсу та наборів перевірки.

In [ ]:
X_train_dropped = X_train.dropna(axis='rows')
y_train_dropped = y_train.loc[X_train_dropped.index]
X_val_dropped = X_val.dropna(axis='rows')
y_val_dropped = y_val.loc[X_val_dropped.index]

<a name='5'></a>
## 5. Дерева рішень
Дерева рішень у машинному навчанні використовуються як передбачувальні моделі, що відображають знання про об'єкт (представлені гілками) у множину рішень. Це один з підходів до передбачувального моделювання у статистиці, добуванні даних та машинному навчанні.
Розповівши про дерева рішень, ми вирішили використовувати класифікатор дерева рішень. Використовуйте scikit-learn, щоб створити дерево рішень для набору даних лікарні пацієнтів які обстежувались упродовж 10 років та за допомогою вибраного навчального набору даних побудувати дерево рішень з прогнозом.

In [ ]:
dt = DecisionTreeClassifier(max_depth=None, random_state=10)
dt.fit(X_train_dropped, y_train_dropped)

Далі ми оцінимо нашу модель. Ми будемо використовувати C-індекс для оцінки.

Пам’ятаєте що C-індекс важливий числовий показник, що оцінює здатність моделі диференціювати різні класи шляхом кількісного визначення того, як часто, розглядаючи всі пари пацієнтів (A, B), модель каже, що пацієнт A має вищу оцінка ризику, ніж пацієнт B, коли, згідно з даними спостереження, пацієнт A фактично помер, а пацієнт B фактично живий. У нашому випадку наша модель є двійковим класифікатором, де кожна оцінка ризику дорівнює 1 (модель передбачає, що пацієнт помре), а 0 (що пацієнт буде жити).

Більш формально, визначення допустимих пар пацієнтів як пари з різними результатами, узгоджених пар як допустимих пар, де пацієнт, який помер, мав вищий бал ризику (тобто наша модель передбачила 1 для пацієнта, який помер, і 0 для того, хто живий) , і зв’язки як допустимі пари, де показники ризику були рівними (тобто наша модель передбачила 1 для обох пацієнтів або 0 для обох пацієнтів), C-індекс дорівнює:

C-Index=#конкордантні пари+0,5×#зв'язки#допустимі пари

Запустіть наступну комірку, щоб обчислити C-індекс для курсу та набору перевірки (цього разу ми надали вам реалізацію).

In [ ]:
y_train_preds = dt.predict_proba(X_train_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_preds)}")


y_val_preds = dt.predict_proba(X_val_dropped)[:, 1]
print(f"Val C-Index: {cindex(y_val_dropped.values, y_val_preds)}")

На жаль, здається, що наше дерево перебудоване: воно настільки точно відповідає навчальним даним, що погано узагальнює інші зразки, такі як ті з набору перевірки.

Навчальний C-індекс виходить рівним 1,0, оскільки під час ініціалізації DecisionTreeClasifier ми залишили max_depth і min_samples_split невизначеними. Таким чином, побудоване дерево рішень буде продовжувати розщеплюватись настільки далеко, наскільки це можливо, що майже гарантує точну відповідність навчальним даним.

Щоб впоратися з цим, ви можете змінити деякі гіперпараметри даного дерева.

<a name='Ex-2'></a>
### Вправа 2
Спробуйте знайти набір гіперпараметрів, який покращує узагальнення набору перевірки, і повторно обчисліть C-індекс. Якщо ви все зробите правильно, ви повинні отримати C-індекс вище 0,6 для набору перевірки.

Ви можете звернутися до документації для sklearn DecisionTreeClassifier.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Try limiting the depth of the tree (<code>'max_depth'</code>).</li>
</ul>
</p>

In [ ]:
# Experiment with different hyperparameters for the DecisionTreeClassifier
# until you get a c-index above 0.6 for the validation set
dt_hyperparams = {
    # set your own hyperparameters below, such as 'min_samples_split': 1

    ### START CODE HERE ###

    'max_depth': 3,

    ### END CODE HERE ###
}

Запустіть наступну клітинку, щоб підібрати та оцінити регульоване дерево.

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
dt_reg = DecisionTreeClassifier(**dt_hyperparams, random_state=10)
dt_reg.fit(X_train_dropped, y_train_dropped)

y_train_preds = dt_reg.predict_proba(X_train_dropped)[:, 1]
y_val_preds = dt_reg.predict_proba(X_val_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_preds)}")
print(f"Val C-Index (expected > 0.6): {cindex(y_val_dropped.values, y_val_preds)}")

Якщо ви використали низьку max_depth, ви можете надрукувати все дерево. Це забезпечує легку інтерпретацію. Запустіть наступну клітинку, щоби побудувати дерево рішень.
Дерево рішень є дуже схожим на блок-схему. Використовуючи блок-схему, ви починаєте з початкової точки або кореня діаграми, а потім, залежно від того, як ви відповідаєте критеріям фільтрації цього початкового вузла, ви переходите до одного з наступних можливих вузлів. Цей процес повторюється, поки не буде досягнуто потрібного результату.

In [ ]:
dot_data = StringIO()
export_graphviz(dt_reg, feature_names=X_train_dropped.columns, out_file=dot_data,
                filled=True, rounded=True, proportion=True, special_characters=True,
                impurity=False, class_names=['neg', 'pos'], precision=2)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

> **Overfitting, underfitting, and the bias-variance tradeoff**

>Переобладнання, недообладнання та компроміс зсуву

Алгоритми для дерев рішень
Дерева рішень працюють на основі алгоритмічного підходу, який розбиває набір даних на окремі точки даних на основі різних критеріїв. Ці розбиття виконуються за допомогою різних змінних або різних функцій набору даних.
Рішення дерев часто корисні коли необхідно провести класифікацію, але основним обмеженням є час обчислення. Дерева рішень можуть чітко визначити, які функції у вибраних наборах даних мають найбільшу передбачувану силу. Крім того, на відміну від багатьох алгоритмів машинного навчання, де правила, що використовуються для класифікації даних, важко інтерпретувати, дерева рішень можуть відтворювати інтерпретовані правила.
Якщо ви протестували кілька значень max_depth, ви, можливо, побачили, що значення 3 дає C-індекси навчання та перевірки приблизно 0,689 та 0,630, а max_depth 2 дає кращу узгодженість із значеннями приблизно 0,653 та 0,607. В останньому випадку ми ще більше зменшили переобладнання ціною незначної втрати прогнозної продуктивності.

Порівняйте це зі значенням max_depth 1, що призводить до C-індексів приблизно 0,597 для навчального набору та 0,598 для набору перевірки: ми усунули надмірне оснащення, але зі значно сильнішим погіршенням прогнозної продуктивності.

Нижча прогностична продуктивність на наборах для навчання та тестування вказує на те, що модель недостатньо підходить для даних: вона не вчиться достатньо з навчальних даних і не може узагальнити невидимі дані (тестувальну вибірку в нашому випадку).

Пошук моделі, яка мінімізує та прийнятно врівноважує незбалансованість та налаштування (наприклад, вибір моделі з max_depth 2 порівняно з іншими значеннями) є поширеною проблемою в машинному навчанні, яка відома як компроміс зміщення-дисперсії.

<a name='6'></a>
## 6. Випадкові ліси

Random forest (англ. випадковий ліс) — ансамблевий метод машинного навчання для класифікації, регресії та інших завдань, який працює за допомогою побудови численних дерев прийняття рішень під час тренування моделі й продукує моду для класів (класифікацій) або усереднений прогноз (регресія) побудованих дерев.

Незалежно від того, як ви обираєте гіперпараметри, одне дерево рішень схильне до перетворення. Щоб вирішити цю проблему, ви можете спробувати випадкові ліси, які поєднують прогнози з багатьох різних дерев для створення надійного класифікатора.

Як і раніше, ми будемо використовувати scikit-learn для створення випадкового лісу для даних. Ми будемо використовувати гіперпараметри за замовчуванням.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train_dropped, y_train_dropped)

Тепер обчисліть і повідомте C-індекс для випадкового лісу в наборі для навчання та перевірки.

In [ ]:
y_train_rf_preds = rf.predict_proba(X_train_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_rf_preds)}")

y_val_rf_preds = rf.predict_proba(X_val_dropped)[:, 1]
print(f"Val C-Index: {cindex(y_val_dropped.values, y_val_rf_preds)}")

Навчання створення випадкового лісу з гіперпараметрами за замовчуванням призводить до моделі, яка має кращу прогностичну продуктивність, ніж окремі дерева рішень, як у попередньому розділі, але ця модель є надмірною.

Тому нам потрібно налаштувати (або оптимізувати) гіперпараметри, щоб знайти модель, яка має хорошу прогностичну продуктивність і мінімізує переобладнання.

Гіперпараметри, які ми виберемо для налаштування, будуть такими:

n_estimators: кількість дерев, які використовуються в лісі.
max_depth: максимальна глибина кожного дерева.
min_samples_leaf: мінімальна кількість (якщо int) або частка (якщо float) зразків у аркуші.
Підхід, який ми застосовуємо для налаштування гіперпараметрів, відомий як пошук по сітці:

Ми визначаємо набір можливих значень для кожного з цільових гіперпараметрів.

Модель навчається та оцінюється для кожної можливої ​​комбінації гіперпараметрів.

Повертається найефективніший набір гіперпараметрів.

Комірка нижче реалізує пошук у сітці гіперпараметрів із використанням C-індексу для оцінки кожної перевіреної моделі.

In [ ]:
def holdout_grid_search(clf, X_train_hp, y_train_hp, X_val_hp, y_val_hp, hyperparams, fixed_hyperparams={}):
    '''
    Conduct hyperparameter grid search on hold out validation set. Use holdout validation.
    Hyperparameters are input as a dictionary mapping each hyperparameter name to the
    range of values they should iterate over. Use the cindex function as your evaluation
    function.

    Input:
        clf: sklearn classifier
        X_train_hp (dataframe): dataframe for training set input variables
        y_train_hp (dataframe): dataframe for training set targets
        X_val_hp (dataframe): dataframe for validation set input variables
        y_val_hp (dataframe): dataframe for validation set targets
        hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                            names to range of values for grid search
        fixed_hyperparams (dict): dictionary of fixed hyperparameters that
                                  are not included in the grid search

    Output:
        best_estimator (sklearn classifier): fitted sklearn classifier with best performance on
                                             validation set
        best_hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                                 names to values in best_estimator
    '''
    best_estimator = None
    best_hyperparams = {}

    # hold best running score
    best_score = 0.0

    # get list of param values
    lists = hyperparams.values()

    # get all param combinations
    param_combinations = list(itertools.product(*lists))
    total_param_combinations = len(param_combinations)

    # iterate through param combinations
    for i, params in enumerate(param_combinations, 1):
        # fill param dict with params
        param_dict = {}
        for param_index, param_name in enumerate(hyperparams):
            param_dict[param_name] = params[param_index]

        # create estimator with specified params
        estimator = clf(**param_dict, **fixed_hyperparams)

        # fit estimator
        estimator.fit(X_train_hp, y_train_hp)

        # get predictions on validation set
        preds = estimator.predict_proba(X_val_hp)

        # compute cindex for predictions
        estimator_score = cindex(y_val_hp, preds[:,1])

        print(f'[{i}/{total_param_combinations}] {param_dict}')
        print(f'Val C-Index: {estimator_score}\n')

        # if new high score, update high score, best estimator
        # and best params
        if estimator_score >= best_score:
                best_score = estimator_score
                best_estimator = estimator
                best_hyperparams = param_dict

    # add fixed hyperparamters to best combination of variable hyperparameters
    best_hyperparams.update(fixed_hyperparams)

    return best_estimator, best_hyperparams

<a name='Ex-3'></a>
### Вправа 3
У клітинці нижче визначте значення, за якими потрібно запустити пошук сітки гіперпараметрів, і запустіть клітинку, щоб знайти найефективніший набір гіперпараметрів.

Ваше завдання — отримати C-індекс вище 0,6 як на тренуванні, так і на перевірці.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>

In [ ]:
def random_forest_grid_search(X_train_dropped, y_train_dropped, X_val_dropped, y_val_dropped):

    # Define ranges for the chosen random forest hyperparameters
    hyperparams = {

        ### START CODE HERE (REPLACE array values with your code) ###

        # how many trees should be in the forest (int)
        'n_estimators': [100, 150, 200],

        # the maximum depth of trees in the forest (int)

        'max_depth': [3, 4, 5],

        # the minimum number of samples in a leaf as a fraction
        # of the total number of samples in the training set
        # Can be int (in which case that is the minimum number)
        # or float (in which case the minimum is that fraction of the
        # number of training set samples)
        'min_samples_leaf': [3, 4],

        ### END CODE HERE ###
    }


    fixed_hyperparams = {
        'random_state': 10,
    }

    rf = RandomForestClassifier

    best_rf, best_hyperparams = holdout_grid_search(rf, X_train_dropped, y_train_dropped,
                                                    X_val_dropped, y_val_dropped, hyperparams,
                                                    fixed_hyperparams)

    print(f"Best hyperparameters:\n{best_hyperparams}")


    y_train_best = best_rf.predict_proba(X_train_dropped)[:, 1]
    print(f"Train C-Index: {cindex(y_train_dropped, y_train_best)}")

    y_val_best = best_rf.predict_proba(X_val_dropped)[:, 1]
    print(f"Val C-Index: {cindex(y_val_dropped, y_val_best)}")

    # add fixed hyperparamters to best combination of variable hyperparameters
    best_hyperparams.update(fixed_hyperparams)

    return best_rf, best_hyperparams

In [ ]:
best_rf, best_hyperparams = random_forest_grid_search(X_train_dropped, y_train_dropped, X_val_dropped, y_val_dropped)

Нарешті, оцініть модель на тестовому наборі. Це важливий крок, оскільки випробування багатьох комбінацій гіперпараметрів і їх оцінка на перевірочному наборі може призвести до того, що модель переповнить перевірочний набір. Тому нам потрібно перевірити, чи добре модель працює на невидимих ​​даних, що є роллю тестового набору, яку ми виконували досі.

In [ ]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
y_test_best = best_rf.predict_proba(X_test)[:, 1]

print(f"Test C-Index: {cindex(y_test.values, y_test_best)}")

Your C-Index on the test set should be greater than `0.6`.

<a name='7'></a>
## 7. Імпутація
Тепер ви створили й оптимізували модель випадкового лісу на основі наших даних. Проте падіння тестового індексу C все одно спостерігалося. Можливо, ви викинули більше половини наших даних через відсутність значень систолічного артеріального тиску. Натомість ми можемо спробувати заповнити або врахувати ці значення.

По-перше, давайте дослідимо, чи наші дані випадково відсутні чи ні. Давайте побудуємо гістограми випущених рядків проти кожної з коваріат (окрім систолічного артеріального тиску), щоб побачити, чи є тенденція. Порівняйте їх із гістограмами об’єкта в усьому наборі даних. Спробуйте побачити, чи одна з коваріат має суттєво різний розподіл у двох підмножинах.

In [ ]:
dropped_rows = X_train[X_train.isnull().any(axis=1)]

columns_except_Systolic_BP = [col for col in X_train.columns if col not in ['Systolic BP']]

for col in columns_except_Systolic_BP:
    sns.distplot(X_train.loc[:, col], norm_hist=True, kde=False, label='full data')
    sns.distplot(dropped_rows.loc[:, col], norm_hist=True, kde=False, label='without missing data')
    plt.legend()

    plt.show()

Більшість коваріатів розподіляються подібним чином незалежно від того, чи ми відкинули рядки з відсутніми даними. Іншими словами, відсутність даних не залежить від цих коваріатів.

Якби це було правдою для всіх коваріантів, тоді можна було б сказати, що дані повністю відсутні (MCAR).

Але, розглядаючи коваріату віку, ми бачимо, що набагато більше даних, як правило, не вистачає для пацієнтів старше 65 років. Причина може полягати в тому, що артеріальний тиск вимірювався рідше для літніх людей, щоб уникнути додаткового навантаження на них.

Оскільки відсутність пов’язана з однією або декількома коваріантами, відсутні дані називаються випадковими (MAR).

Проте, виходячи з наявної у нас інформації, немає підстав вважати, що значення відсутніх даних — або, зокрема, значення відсутніх систолічних артеріальних тисків — пов’язані з віком пацієнтів. Якби це було так, тоді ці дані вважалися б відсутніми не випадково (MNAR).

<a name='8'></a>
## 8. Аналіз помилок

Вправа 4
Використовуючи інформацію з наведених вище графіків, спробуйте знайти підгрупу тестових даних, на якій модель працює погано. Ви повинні легко знайти підгрупу з принаймні 250 випадків, для яких модель має C-індекс менше 0,69.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Define a mask using a feature and a threshold, e.g. patients with a BMI below 20: <code>mask = X_test['BMI'] < 20 </code>. </li>
    <li> Try to find a subgroup for which the model had little data.</li>
</ul>
</p>

In [ ]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def bad_subset(forest, X_test, y_test):
    # define mask to select large subset with poor performance
    # currently mask defines the entire set

    ### START CODE HERE (REPLACE the code after 'mask =' with your code) ###
    mask = X_test.Age < 40
    ### END CODE HERE ###

    X_subgroup = X_test[mask]
    y_subgroup = y_test[mask]
    subgroup_size = len(X_subgroup)

    y_subgroup_preds = forest.predict_proba(X_subgroup)[:, 1]
    performance = cindex(y_subgroup.values, y_subgroup_preds)

    return performance, subgroup_size

#### Перевірте свою роботу

In [ ]:
performance, subgroup_size = bad_subset(best_rf, X_test, y_test)
print("Subgroup size should greater than 250, performance should be less than 0.69 ")
print(f"Subgroup size: {subgroup_size}, C-Index: {performance}")

#### Очікуваний результат
Зауважте, ваш фактичний вихід буде відрізнятися залежно від вибраних гіперпараметрів і вибраної маски.

Переконайтеся, що c-індекс менше 0,69
Розмір підгрупи: 586, C-індекс: 0,6275
```

**Bonus**:
- See if you can get a c-index as low as 0.53
```
Subgroup size: 251, C-Index: 0.5331
```

<a name='9'></a>
## 9. Підходи імпутації
Переконавшись, що наші дані не пропадають абсолютно випадково, ми можемо обробити відсутні значення, замінивши їх заміненими значеннями на основі інших значень, які у нас є. Це відомо як імпутація.

Першою стратегією імпутації, яку ми будемо використовувати, є заміна середнього: ми замінимо відсутні значення для кожної ознаки середнім із доступних значень. У наступній комірці скористайтеся SimpleImputer від sklearn, щоб використовувати середнє імпутування для відсутніх значень.

In [ ]:
# Impute values using the mean
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train_mean_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val_mean_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

<a name='Ex-5'></a>
### Вправа 5
Тепер виконайте пошук у сітці гіперпараметрів, щоб знайти найефективнішу модель випадкового лісу, і повідомте про результати для тестового набору.

Визначте діапазони параметрів для пошуку гіперпараметрів у наступній клітинці та запустіть клітинку.

Цільова продуктивність
Зробіть свій індекс с принаймні 0,74 або вище

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>


In [ ]:
# Define ranges for the random forest hyperparameter search
hyperparams = {
    ### START CODE HERE (REPLACE array values with your code) ###

    # how many trees should be in the forest (int)
    'n_estimators': [100, 150, 200],

    # the maximum depth of trees in the forest (int)
    'max_depth': [3, 4, 5],

    # the minimum number of samples in a leaf as a fraction
    # of the total number of samples in the training set
    # Can be int (in which case that is the minimum number)
    # or float (in which case the minimum is that fraction of the
    # number of training set samples)
    'min_samples_leaf': [3, 4],

    ### END CODE HERE ###
}

In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
rf = RandomForestClassifier

rf_mean_imputed, best_hyperparams_mean_imputed = holdout_grid_search(rf, X_train_mean_imputed, y_train,
                                                                     X_val_mean_imputed, y_val,
                                                                     hyperparams, {'random_state': 10})

print("Performance for best hyperparameters:")

y_train_best = rf_mean_imputed.predict_proba(X_train_mean_imputed)[:, 1]
print(f"- Train C-Index: {cindex(y_train, y_train_best):.4f}")

y_val_best = rf_mean_imputed.predict_proba(X_val_mean_imputed)[:, 1]
print(f"- Val C-Index: {cindex(y_val, y_val_best):.4f}")

y_test_imp = rf_mean_imputed.predict_proba(X_test)[:, 1]
print(f"- Test C-Index: {cindex(y_test, y_test_imp):.4f}")

#### Очікуваний результат
Зауважте, що ваші фактичні значення с-індексу відрізнятимуться залежно від вибраних гіперпараметрів.

Спробуйте отримати хороший тест c-index, подібний до цих цифр нижче:
```Python
Performance for best hyperparameters:
- Train C-Index: 0.8109
- Val C-Index: 0.7495
- Test C-Index: 0.7805
```

Далі ми застосуємо іншу стратегію імпутації, відому як багатовимірна імпутація ознак, використовуючи клас IterativeImputer scikit-learn (див. документацію).

За допомогою цієї стратегії для кожної функції, для якої відсутні значення, регресійна модель навчена передбачати спостережувані значення на основі всіх інших ознак, а відсутні значення виводяться за допомогою цієї моделі. Оскільки однієї ітерації для всіх функцій може бути недостатньо для імпутації всіх відсутніх значень, може бути виконано кілька ітерацій, звідси й назва класу IterativeImputer.

У наступній комірці використовуйте IterativeImputer, щоб виконати багатовимірне імпутування ознак.

Зауважте, що під час першого запуску клітинки imputer.fit(X_train) може виникнути помилка з повідомленням LinAlgError: SVD did not converge: просто повторно запустіть клітинку.

In [ ]:
# Impute using regression on other covariates
imputer = IterativeImputer(random_state=0, sample_posterior=False, max_iter=1, min_value=0)
imputer.fit(X_train)
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

<a name='Ex-6'></a>
### Вправа 6
Виконайте пошук у сітці гіперпараметрів, щоб знайти найефективнішу випадкову модель лісу, і повідомте про результати для тестового набору. Визначте діапазони параметрів для пошуку гіперпараметрів у наступній клітинці та запустіть клітинку.

Цільова продуктивність
Спробуйте отримати c-індекс тексту принаймні 0,74 або вище.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>


In [ ]:
# Define ranges for the random forest hyperparameter search
hyperparams = {
    ### START CODE HERE (REPLACE array values with your code) ###

    # how many trees should be in the forest (int)
    'n_estimators': [100, 150, 200],

    # the maximum depth of trees in the forest (int)
    'max_depth': [3, 4, 5],

    # the minimum number of samples in a leaf as a fraction
    # of the total number of samples in the training set
    # Can be int (in which case that is the minimum number)
    # or float (in which case the minimum is that fraction of the
    # number of training set samples)
    'min_samples_leaf': [3, 4],

    ### END CODE HERE ###
}

Двічі клацніть (або введіть), щоб редагувати

In [ ]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
rf = RandomForestClassifier

rf_imputed, best_hyperparams_imputed = holdout_grid_search(rf, X_train_imputed, y_train,
                                                           X_val_imputed, y_val,
                                                           hyperparams, {'random_state': 10})

print("Performance for best hyperparameters:")

y_train_best = rf_imputed.predict_proba(X_train_imputed)[:, 1]
print(f"- Train C-Index: {cindex(y_train, y_train_best):.4f}")

y_val_best = rf_imputed.predict_proba(X_val_imputed)[:, 1]
print(f"- Val C-Index: {cindex(y_val, y_val_best):.4f}")

y_test_imp = rf_imputed.predict_proba(X_test)[:, 1]
print(f"- Test C-Index: {cindex(y_test, y_test_imp):.4f}")

#### Очікуваний результат
Зауважте, ваш фактичний вихід буде відрізнятися залежно від вибраних гіперпараметрів і вибраної маски.

Продуктивність для найкращих гіперпараметрів:
- Індекс C поїзда: 0,8131
- Val C-індекс: 0,7454
- Тест C-індекс: 0,7797

<a name='10'></a>
## 10. Порівняння
Для хорошої міри повторіть перевірку на попередній підгрупі, щоб побачити, чи ваші нові моделі працюють краще.

In [ ]:
performance, subgroup_size = bad_subset(best_rf, X_test, y_test)
print(f"C-Index (no imputation): {performance}")

performance, subgroup_size = bad_subset(rf_mean_imputed, X_test, y_test)
print(f"C-Index (mean imputation): {performance}")

performance, subgroup_size = bad_subset(rf_imputed, X_test, y_test)
print(f"C-Index (multivariate feature imputation): {performance}")

Ми повинні побачити, що уникнення повного аналізу випадків (тобто аналізу лише спостережень, для яких немає відсутніх даних) дозволяє нашій моделі трохи краще узагальнювати. Не забувайте досліджувати свої відсутні кейси, щоб визначити, чи вони відсутні випадковим чином чи ні!

<a name='11'></a>
## 11. Пояснення: SHAP
Використання випадкового лісу покращило результати, але ми втратили частину природної інтерпретації дерев. У цьому розділі ми спробуємо пояснити прогнози за допомогою трохи складніших методів.

Ви вирішили застосувати *SHAP (додаткові пояснення SHapley) *, передовий метод, який пояснює прогнози, зроблені моделями машинного навчання чорної скриньки (тобто моделями, які є надто складними, щоб бути зрозумілими людьми).

З огляду на передбачення, зроблене моделлю машинного навчання, значення SHAP пояснюють передбачення кількісним визначенням додаткової важливості кожної функції для прогнозу. Значення SHAP беруть свій початок у теорії кооперативних ігор, де значення використовуються для кількісної оцінки внеску кожного гравця в гру.

Незважаючи на те, що обчислювати значення SHAP для загальних моделей чорної скриньки дорого, у випадку дерев і лісів існує швидкий поліноміальний алгоритм. Щоб отримати докладнішу інформацію, перегляньте статтю TreeShap.

Ми використаємо бібліотеку shap, щоб зробити це для нашої моделі випадкового лісу. Запустіть наступну клітинку, щоб вивести осіб із найбільшим ризиком у тестовому наборі відповідно до нашої моделі.

In [ ]:
X_test_risk = X_test.copy(deep=True)
X_test_risk.loc[:, 'risk'] = rf_imputed.predict_proba(X_test_risk)[:, 1]
X_test_risk = X_test_risk.sort_values(by='risk', ascending=False)
X_test_risk.head()

Ми можемо використовувати значення SHAP, щоб спробувати зрозуміти результати моделі на конкретних індивідуумах, використовуючи графіки сил. Проведіть по клітинці нижче, щоб побачити графік впливу на найбільш ризиковану особу.

In [ ]:
explainer = shap.TreeExplainer(rf_imputed)
i = 0
shap_value = explainer.shap_values(X_test.loc[X_test_risk.index[i], :])[1]
shap.force_plot(explainer.expected_value[1], shap_value, feature_names=X_test.columns, matplotlib=True)

Як читати цю таблицю:

Червоні секції ліворуч — це функції, які підштовхують модель до кінцевого прогнозу в позитивному напрямку (тобто вищий вік збільшує прогнозований ризик).
Сині розділи праворуч — це функції, які штовхають модель до кінцевого прогнозу в негативному напрямку (якщо збільшення функції призводить до зниження ризику, вона буде показана синім кольором).
Зауважте, що точні результати вашої діаграми відрізнятимуться залежно від гіперпараметрів, які ви виберете для своєї моделі.
Ми також можемо використовувати значення SHAP, щоб зрозуміти результат моделі в сукупності. Запустіть наступну клітинку, щоб ініціалізувати значення SHAP (це може зайняти кілька хвилин).

In [ ]:
shap_values = shap.TreeExplainer(rf_imputed).shap_values(X_test)[1]

Запустіть наступну клітинку, щоб побачити підсумковий графік значень SHAP для кожної функції в кожному з тестових прикладів. Кольори вказують на значення функції.

In [ ]:
shap.summary_plot(shap_values, X_test, show=False)

Ми чітко бачимо, що бути жінкою (стать = 2,0, на відміну від чоловіків, для яких стать = 1,0) має негативне значення SHAP, тобто знижує ризик смерті протягом 10 років. Високий вік і високий систолічний артеріальний тиск мають позитивні значення SHAP і, отже, пов’язані з підвищеною смертністю.

Ви можете побачити, як елементи взаємодіють, використовуючи графіки залежностей. Вони відображають значення SHAP для даної функції для кожної точки даних і розфарбовують точки, використовуючи значення для іншої функції. Це дозволяє нам почати пояснювати варіацію значення SHAP для одного значення основної функції.

Запустіть наступну клітинку, щоб побачити взаємодію між віком і статтю.

In [ ]:
shap.dependence_plot('Age', shap_values, X_test, interaction_index='Sex')

Ми бачимо, що хоча вік > 50 років, як правило, погано (позитивне значення SHAP), бути жінкою, як правило, зменшує вплив віку. Це має сенс, оскільки ми знаємо, що жінки зазвичай живуть довше за чоловіків.

Давайте тепер подивимося на індекс бідності та вік.

In [ ]:
shap.dependence_plot('Poverty index', shap_values, X_test, interaction_index='Age')

Ми бачимо, що вплив індексу бідності швидко спадає, а для людей з вищим доходом вік починає пояснювати значну частину варіацій у впливі індексу бідності.

Спробуйте інші пари та подивіться, які ще цікаві стосунки ви можете знайти!

# Щиро вітаю!
Ви виконали цікаве завдання з даного курсу. У даній роботі ви навчилися складати дерева рішень, випадкові ліси та працювати з відсутніми даними.
Дерева рішень - це важливий інструмент в прийнятті рішень, який використовується у багатьох галузях, включаючи бізнес, науку, технології, медицину та інші. Основна мета дерев рішень - систематизувати інформацію та визначити оптимальний варіант дії в умовах невизначеності або складності.